# Supervised learning- categorisation- logistic regression

# 0. Loading packages and dataset

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-darkgrid')

df = pd.read_csv('real_estate_cleaned.csv')

We will use the same dataset as for the regression setting. But now, we will not try to predict the class, but the price class. In the following code, I create two price classes 
+ 0 = all prices below 400 000 (or the log(prices) < 12.9 - remember that we used the log-transform when cleaning the data)
+ 1 = all prices above 400 000


In [ ]:
#df['price_class'] = 1
#df['price_class'][df.tx_price > 12.9] = 2
#df['price_class'][df.tx_price > 13.3] = 3

df['price_class'] = 0
df.loc[df.tx_price > 12.9, 'price_class'] = 1
#df['price_class'][df.tx_price > 12.9] = 1


In [ ]:
df[['price_class','tx_price']]

Let's see how many observations we have in each class

In [ ]:
df['price_class'].value_counts()

Clearly imbalanced. We will see how to deal with this later on.

In [ ]:
df_new = df.drop(['tx_price'],1)

In [ ]:
df_new.head()

Note that we only did this for instruction reasons. You do not want to classify your outcome like this if you have continuous data. An algoritm will always be able to get more information out of the continuous data. Say you are only interested in knowing if a house is more expensive than 500 000 euro or not. Then it is a better strategy to predict the continuous prices and then classify after prediction). 

## 0.1. Train/val/test-split
We make a three-way split, because we need an extra hold-out set for the calibration.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X = df_new.drop(['price_class'],1)
y = df_new['price_class']

X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
X_train, X_val, y_train, y_val = train_test_split(X_trainval,y_trainval, test_size=0.2, random_state=81357)


num_feat = ['beds', 'baths', 'sqft', 'year_built', 'lot_size', 'restaurants',
       'groceries', 'nightlife', 'cafes', 'shopping', 'arts_entertainment',
       'beauty_spas', 'active_life', 'median_age', 'married', 'college_grad',
       'property_tax', 'insurance', 'median_school', 'num_schools', 'tx_year',
       'school_score', 'tax_per_sqft']
scaler = StandardScaler()
X_train_stand = X_train.copy()
X_trainval_stand = X_trainval.copy()
X_val_stand = X_val.copy()

X_test_stand = X_test.copy()
X_train_stand[num_feat] = scaler.fit_transform(X_train_stand[num_feat])
X_val_stand[num_feat] = scaler.transform(X_val_stand[num_feat])
X_trainval_stand[num_feat] = scaler.transform(X_trainval_stand[num_feat])
X_test_stand[num_feat] = scaler.transform(X_test_stand[num_feat])

In [ ]:
X_train.head()

# 1. Logistic regression
 ## 1.1 Basic model

We start with the simple logistic regression model. The C-parameter (the inverse of the regularisation) is per default at 1, which means there is regularisation. To have no regularisation, we put C at a very high number (10 000). Since we have a validation set, we can evaluate on that one, whitout risking bias

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=10000, max_iter=200) 
logreg.fit(X_train_stand, y_train)
print(logreg.score(X_train_stand, y_train))
print(logreg.score(X_val_stand, y_val))


The score object gives the accuracy for classification (while it gives the R2 for regression problems). The predictions can also be asked as before. You can choose between .predict(), which gives the predicted class and .predict_proba(), which gives the predicted chance for each class.

In [ ]:
y_val_pred = logreg.predict(X_val_stand)

In [ ]:
y_val_pred_prob = logreg.predict_proba(X_val_stand)

In [ ]:
#!pip install scikit-plot

We can also take a look at the confusion matrix.

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_val, y_val_pred)
print(confusion_matrix)

import scikitplot as skplt
skplt.metrics.plot_confusion_matrix(y_val, y_val_pred, normalize=True)
plt.show()

You could now calculate the quality measure you want yourself. Or you can get some specific quality metrics from a classification report.


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_val_pred))

+ The micro-average is the accuracy.
+ The macro average is the average over the three classes for each specific measure
+ The weighted average is the same as the macro average, but weighted for the number of observations in each class.

Notice the high recall for the first class, while the recall of the second class is very low. This might be because of the imbalanced nature of our dataset. There are a lot of observations in class 1. We should take this into account (but we will see how in the next class). We will first focus on calibration.


In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
preds = y_val_pred_prob[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_val, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

## 1.2 Calibration

Let's check if our model is wel calibrated, and if not, perform a calibration.

In [ ]:
from sklearn.calibration import calibration_curve
y_pred_val_prob = logreg.predict_proba(X_val_stand)

fop, mpv = calibration_curve(y_val,y_pred_val_prob[:,1],n_bins=10)

plt.plot(mpv,fop, marker='o', linewidth=1, label='uncalibrated')
plt.plot([0,1],[0,1],linestyle='--', label='perfectly calibrated')
plt.xlabel('Mean predicted probability')
plt.ylabel('Fraction of positives')
plt.legend()
plt.show()

This is not good, but not teribly bad either (normally, logistic model tend to be well calibrated, calibration is more needed for tree-based methods, support vector machines...). We will do a isotonic regression to correct this. Note that we use the validationset to fit the isotonic regression. We set cv at 'prefit', because we have already fitted the model. So, the function assumes it can use the entire input dataset for validation.

In [ ]:
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
isotonic = CalibratedClassifierCV(logreg, cv='prefit', method='isotonic')
isotonic.fit(X_val_stand, y_val)

In [ ]:
from sklearn.calibration import calibration_curve

y_val_pred_prob_c = isotonic.predict_proba(X_val_stand)

fop, mpv = calibration_curve(y_val,y_pred_val_prob[:,1],n_bins=10)
fop_c, mpv_c = calibration_curve(y_val,y_val_pred_prob_c[:,1],n_bins=10)

plt.plot(mpv,fop, marker='o', linewidth=1, label='uncalibrated')
plt.plot(mpv_c,fop_c, marker='o', linewidth=1, label='calibrated')

plt.plot([0,1],[0,1],linestyle='--', label='perfectly calibrated')

plt.xlabel('Mean predicted probability')
plt.ylabel('Fraction of positives')

plt.legend()
plt.show()

Now, this is much better (do not expect you calibration to end up to be perfect like this, especially not for other algorithms.)

In [ ]:
y_pred = logreg.predict(X_val_stand)
y_pred_c = isotonic.predict(X_val_stand)

print(classification_report(y_val, y_pred))
print(classification_report(y_val, y_pred_c))

We see a good recal for the first class, but a drop for the second class. You actually don't have to do the fitting of the model and the calibration seperately. You can do this at once (and it is better), using the CalibratedClassifierCV, but now you give a value for CV (e.g 3 or 5) instead of 'prefit'. A cross-validation is performed, where the train-set is split into 2, one for training and one for calibration. Since we want to compare several algorithms, we keep the validation dataset seperate to evaluate on (if you choose one algorithm, you can use the complete train_val dataset to do training and calibration on). 

In [ ]:
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
logreg2 = LogisticRegression(C=10000, max_iter=1000) 
isotonic2 = CalibratedClassifierCV(logreg2, cv=3, method='isotonic')
isotonic2.fit(X_train_stand, y_train)


# The calibration plot
y_val_pred_prob_c2 = isotonic2.predict_proba(X_val_stand)

fop, mpv = calibration_curve(y_val,y_pred_val_prob[:,1],n_bins=10)
fop_c, mpv_c = calibration_curve(y_val,y_val_pred_prob_c2[:,1],n_bins=10)

plt.plot(mpv,fop, marker='o', linewidth=1, label='uncalibrated')
plt.plot(mpv_c,fop_c, marker='o', linewidth=1, label='calibrated')
plt.plot([0,1],[0,1],linestyle='--', label='perfectly calibrated')

plt.xlabel('Mean predicted probability')
plt.ylabel('Fraction of positives')

plt.legend()
plt.show()

y_pred_c2 = isotonic2.predict(X_val_stand)
print(classification_report(y_val, y_pred_c))
print(classification_report(y_val, y_pred_c2))

While this calibration seems less good in the plot, it is a much more realistic picture. It isn't perfect on the line, because there have been 3 seperate calibration, one for each test-fold within the cross-validation, resulting in three models. At the end, the predicted chance is the average of the three models. This is an example of ensemble learning.



## 1.3 Imbalanced data
The dataset is imbalanced: there are a lot of observations in class 1.We should take this into account. We could use over- or undersampling. But logistic regression has a built-in argument to do class-weight balancing. So, we will use that.

In [ ]:
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
logreg_balanced = LogisticRegression(C=10000, class_weight='balanced', max_iter=1000) 
isotonic3 = CalibratedClassifierCV(logreg_balanced, cv=3, method='isotonic')
isotonic3.fit(X_train_stand, y_train)

# The calibration plot
y_val_pred_prob_c3 = isotonic3.predict_proba(X_val_stand)

fop, mpv = calibration_curve(y_val,y_pred_val_prob[:,1],n_bins=10)
fop_c, mpv_c = calibration_curve(y_val,y_val_pred_prob_c2[:,1],n_bins=10)
fop_c3, mpv_c3 = calibration_curve(y_val,y_val_pred_prob_c3[:,1],n_bins=10)

plt.plot(mpv,fop, marker='o', linewidth=1, label='uncalibrated')
plt.plot(mpv_c,fop_c, marker='o', linewidth=1, label='calibrated')
plt.plot(mpv_c3,fop_c3, marker='o', linewidth=1, label='calibrated balanced')

plt.plot([0,1],[0,1],linestyle='--', label='perfectly calibrated')

plt.xlabel('Mean predicted probability')
plt.ylabel('Fraction of positives')

plt.legend()
plt.show()

y_pred_c3 = isotonic3.predict(X_val_stand)
print(classification_report(y_val, y_pred_c2))
print(classification_report(y_val, y_pred_c3))

We can see that the recall of the second class improves a bit.

## 1.3 Adding polynomials and penalisation

Let's add polynomials, but immediately do the penalisation as well (since sklearn does this automatically anyway, using l2). We will go up to the 3th order of polynomials and then regulate it back down using penalisation (you could go to a higher order, but I choose 3 to keep the computation time low for instruction reasons). If you have more time to run the model, try it out and see what happens if you increase the order.

We will also use cross-validation to find the optimal penalisation factor C. We will train on accuracy, since we don't really have a preference for recall or precision here. sklearn picks accuracy automatically, so you actually don't need to specify this, but just to show how to do this in case you want another metric, I show how.



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.metrics import make_scorer

prec_scorer = make_scorer(metrics.accuracy_score)
# other possibilities are metrics.recall_score, metrics.average_precision, ...

C = [round(x,5) for x in np.linspace(start = 0.0001, stop = 10, num = 1000)]
#penalty = ['l1', 'l2']
class_weight = ['balanced']
random_grid = {'C': C,    
               'class_weight': class_weight}

#Designing polynomial features
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3)
X_train_poly = poly.fit_transform(X_train_stand)
X_val_poly = poly.fit_transform(X_val_stand)
X_test_poly = poly.transform(X_test_stand)

# The object to fit the model
logreg = LogisticRegression( max_iter=1000) 

# object for the randomised search
log_random = RandomizedSearchCV(estimator = logreg, param_distributions = random_grid,
                                scoring=prec_scorer, n_iter = 100,
                               cv = 3, verbose=2,  n_jobs=-1)

#Fitting model (model+ calibration)
log_random.fit(X_train_poly, y_train)

In [ ]:
log_random.best_score_

In [ ]:
log_random.best_params_

In [ ]:
out2 = pd.DataFrame(log_random.cv_results_)

#Have to replace the None values with something else, or it wont make the plots
res = [i for i in range(len(out2['param_class_weight'])) if out2['param_class_weight'][i] == None] 
out2['param_class_weight'].iloc[res] = "No"

xlabel_names = ['param_C','param_class_weight']


plt.scatter(out2['param_C'], out2['mean_test_score'], c='blue');


We can see that lower values for C (high regularisation) lead to higher accuracy. We will zoom in on these values. But, we also include the calibration now.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import make_scorer

prec_scorer = make_scorer(metrics.accuracy_score)
# other possibilities are metrics.recall_score, metrics.average_precision, ...

C = [round(x,5) for x in np.linspace(start = 0, stop =4, num = 100)]
class_weight = ['balanced']

random_grid = {'C': C,
               'class_weight': class_weight}

# The object to fit the model
logreg_balanced = LogisticRegression( max_iter=1000) 

# object for the randomised search
log_grid = GridSearchCV(estimator = logreg_balanced, param_grid = random_grid,
                                scoring=prec_scorer  , cv = 3, verbose=2,  n_jobs=-1)
isotonic_log = CalibratedClassifierCV(log_grid, cv=3, method='isotonic')

#Fitting model (model+ calibration)
isotonic_log.fit(X_train_poly, y_train)

In [ ]:
#isotonic_log.best_params_


This doesn't work, because we do not have 1 best model, but three, because of the cross-validation. So each fold choses its own best model and calibrates this.

In [ ]:
isotonic_log.score(X_val_poly, y_val)

In [ ]:
y_pred_poly = isotonic_log.predict(X_val_poly)
print(classification_report(y_val, y_pred_poly))

In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
preds = y_val_pred_prob[:,1]
preds_poly = isotonic_log.predict_proba(X_val_poly)[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_val, preds)
fpr_p, tpr_p, threshold = metrics.roc_curve(y_val, preds_poly)

roc_auc = metrics.auc(fpr, tpr)
roc_auc_p = metrics.auc(fpr_p, tpr_p)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'logistic: AUC =  %0.2f' % roc_auc)
plt.plot(fpr_p, tpr_p, 'g', label = 'polynomial AUC = %0.2f' % roc_auc_p)

plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

The ROC-curve shows that this final model is better than the simple logistic regression (which was already pretty good.)

# 2. knn


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
n_neighbors = 4*np.arange(1,50)
param_grid = {'n_neighbors': n_neighbors}
knn = KNeighborsClassifier( )
grid_search = GridSearchCV(estimator = knn, param_grid = param_grid, scoring=prec_scorer ,cv = 5,  verbose=2, n_jobs = -1)
grid_search.fit(X_train_stand, y_train)
grid_search.best_params_

In [ ]:
out2 = pd.DataFrame(grid_search.cv_results_)
xlabel_names = ['n_neighbors']
plt.scatter(out2['param_n_neighbors'], out2['mean_test_score'], c='blue');


In [ ]:
y_pred_knn = grid_search.predict(X_val_stand)
print(classification_report(y_val, y_pred_knn))

In [ ]:
preds = y_val_pred_prob[:,1]
preds_poly = isotonic_log.predict_proba(X_val_poly)[:,1]
preds_knn = grid_search.predict_proba(X_val_stand)[:,1]


fpr, tpr, threshold = metrics.roc_curve(y_val, preds)
fpr_p, tpr_p, threshold = metrics.roc_curve(y_val, preds_poly)
fpr_k, tpr_k, threshold = metrics.roc_curve(y_val, preds_knn)

roc_auc = metrics.auc(fpr, tpr)
roc_auc_p = metrics.auc(fpr_p, tpr_p)
roc_auc_k = metrics.auc(fpr_k, tpr_k)


plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'logistic: AUC =  %0.2f' % roc_auc)
plt.plot(fpr_p, tpr_p, 'g', label = 'polynomial AUC = %0.2f' % roc_auc_p)
plt.plot(fpr_k, tpr_k, 'r', label = 'knn AUC = %0.2f' % roc_auc_k)

plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Seems that the polynomial logistic model performed better. However, we did not correct for the imbalance in this model. There is no function class_weight for knn. So we will have to correct the imbalance ourself. We will use hybrid sampling, where we first oversample using smote and then also undersample.

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

over = SMOTE(sampling_strategy=0.8, random_state=1703)
under = RandomUnderSampler(sampling_strategy=1)
pipeline = Pipeline([('o', over), ('u', under)])

X_train_smote, y_train_smote = pipeline.fit_resample(X_train_stand.copy(), y_train.copy())
X_val_smote, y_val_smote = pipeline.fit_resample(X_val_stand.copy(), y_val.copy())
np.bincount(y_val_smote)

we already know lower values of k perform better, so I will focus on the lower values (up to 50 instead of 200 previously)

In [ ]:
n_neighbors = np.arange(1,50)
param_grid = {'n_neighbors': n_neighbors}

knn_b = KNeighborsClassifier( )
grid_search_knn = GridSearchCV(estimator = knn, param_grid = param_grid, scoring=prec_scorer ,cv = 5,  verbose=2, n_jobs = -1)
isotonic_knn = CalibratedClassifierCV(grid_search_knn, cv=3, method='isotonic')


isotonic_knn.fit(X_train_smote, y_train_smote)


In [ ]:
y_pred_knn_b = isotonic_knn.predict(X_val_stand)
print(classification_report(y_val, y_pred_knn_b))

The two results for both recall and precision are now a lot better balanced between the two classes. And the overal accuracy is also higher.

In [ ]:
preds_knn_b = isotonic_knn.predict_proba(X_val_stand)[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_val, preds)
fpr_p, tpr_p, threshold = metrics.roc_curve(y_val, preds_poly)
fpr_k, tpr_k, threshold = metrics.roc_curve(y_val, preds_knn_b)


roc_auc = metrics.auc(fpr, tpr)
roc_auc_p = metrics.auc(fpr_p, tpr_p)
roc_auc_k = metrics.auc(fpr_k, tpr_k)


plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'logistic: AUC =  %0.2f' % roc_auc)
plt.plot(fpr_p, tpr_p, 'g', label = 'polynomial AUC = %0.2f' % roc_auc_p)
plt.plot(fpr_k, tpr_k, 'r', label = 'knn AUC = %0.2f' % roc_auc_k)

plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

But the polynomial model still performs better overal. And according to the ROC-curve the model became worse. You see, you need to decide in advance what you will use to evaluate your model, or you will have conflicting messages. Let's move on to tree-based models.

# 3. GBM
We won't do the simple decision tree or random forest, but just stick to the gradient boosting machine for the tree based methods. Again, it is very similar to the regression setting. Since GBM also does not have a class_weight function, we will use the smoted dataset again. 

We begin with a random search and end with a grid search that includes the calibration.

In [ ]:
 np.linspace(start = 0.001, stop = 1.5, num = 50)
 #np.linspace(1, 10, num = 20)
    

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV


n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 50)]
max_features = ['auto', 'sqrt', 'log2']
max_depth = [int(x) for x in np.linspace(1, 10, num = 10)]
min_samples_split = [int(x) for x in np.linspace(2, 10, num = 9)]
min_samples_leaf = [int(x) for x in np.linspace(1, 10, num = 10)]
learning_rate = [round(x,5) for x in np.linspace(start = 0.001, stop = 1.5, num = 50)]

random_grid = {'n_estimators': n_estimators,
               'learning_rate': learning_rate,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
gbm = GradientBoostingClassifier()
gbm_random = RandomizedSearchCV(estimator = gbm, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=4872, n_jobs = -1)
gbm_random.fit(X_train_smote, y_train_smote)
gbm_random.best_params_

In [ ]:
y_pred_gbm = gbm_random.predict(X_val_stand)
print(classification_report(y_val, y_pred_gbm))

Wow, a high accuracy, with good values for precision and recall for both classes. This must be the best model we have seen so far. Let's see if we can improve it even more in the grid search.

In [ ]:
out2 = pd.DataFrame(gbm_random.cv_results_)

xlabel_names = ['param_max_depth','param_min_samples_split','param_min_samples_leaf','param_n_estimators',
                'param_learning_rate','param_max_features']

fig, axs = plt.subplots(2,3, figsize=(20,10))

axs[0,0].scatter(out2['param_max_depth'], out2['mean_test_score'], c='blue');
axs[0,0].set_title('max_depth')

axs[0,1].scatter(out2['param_min_samples_split'], out2['mean_test_score'], c='blue');
axs[0,1].set_title('min_samples_split')

axs[0,2].scatter(out2['param_min_samples_leaf'], out2['mean_test_score'], c='blue');
axs[0,2].set_title('min_samples_leaf')

axs[1,0].scatter(out2['param_n_estimators'], out2['mean_test_score'], c='blue');
axs[1,0].set_title('n_estimators')

axs[1,1].scatter(out2['param_learning_rate'], out2['mean_test_score'], c='blue');
axs[1,1].set_title('learning_rate')

axs[1,2].scatter(out2['param_max_features'], out2['mean_test_score'], c='blue');
axs[1,2].set_title('max_features')

for ax in axs.flat: ax.set(ylabel='r_squared')

In [ ]:
[x for x in np.linspace(start = 0.2, stop = 0.3, num = 10)]
[x for x in (np.linspace(start = 0.2, stop = 0.3, num = 10), np.linspace(start = 1.2, stop = 1.3, num = 10) )]

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

n_estimators = [797,800,803]
learning_rate = [0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,
                1.2,1.21,1.22,1.23,1.24,1.25,1.26,1.27,1.28,1.29,1.30]
max_features = ['auto']
max_depth = [3,7]
min_samples_split = [9,10]
min_samples_leaf = [5,9]
                                            
random_grid = {'n_estimators': n_estimators,
               'learning_rate': learning_rate,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
gbm = GradientBoostingClassifier()
gbm_grid = GridSearchCV(estimator = gbm, param_grid = random_grid,  cv = 3, verbose=2,  n_jobs = -1)

isotonic_gbm = CalibratedClassifierCV(gbm_grid, cv=3, method='isotonic')
isotonic_gbm.fit(X_train_smote, y_train_smote)


In [ ]:
y_pred_gbm = isotonic_gbm.predict(X_val_stand)
print(classification_report(y_val, y_pred_gbm))

In [ ]:
preds_gbm = isotonic_gbm.predict_proba(X_val_stand)[:,1]


fpr, tpr, threshold = metrics.roc_curve(y_val, preds)
fpr_p, tpr_p, threshold = metrics.roc_curve(y_val, preds_poly)
fpr_k, tpr_k, threshold = metrics.roc_curve(y_val, preds_knn_b)
fpr_g, tpr_g, threshold = metrics.roc_curve(y_val, preds_gbm)

roc_auc = metrics.auc(fpr, tpr)
roc_auc_p = metrics.auc(fpr_p, tpr_p)
roc_auc_k = metrics.auc(fpr_k, tpr_k)
roc_auc_g = metrics.auc(fpr_g, tpr_g)


plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'logistic: AUC =  %0.2f' % roc_auc)
plt.plot(fpr_p, tpr_p, 'g', label = 'polynomial AUC = %0.2f' % roc_auc_p)
plt.plot(fpr_k, tpr_k, 'r', label = 'knn AUC = %0.2f' % roc_auc_k)
plt.plot(fpr_g, tpr_g, 'y', label = 'gbm AUC = %0.2f' % roc_auc_g)

plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

The ROC-curve also agrees this is the best model yet.

# 4. Support vector machines
One final model. While this model is theoretically one of the harder ones, implementing it is rather simple, just do a cross-validation using four hyperparameters.

In [ ]:
np.linspace(start = 1, stop = 10, num = 10)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV

C = [round(x,5) for x in np.linspace(start = 0.001, stop = 10, num = 50)]
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
degree = [round(x) for x in np.linspace(start = 1, stop = 15, num = 15)]
gamma = [round(x,5) for x in np.linspace(start = 0.001, stop = 1, num = 50)]
#gamma = ['auto', 0.3,0.5, 0.7,0.9]

random_grid = {'C': C,
               'kernel': kernel,
               'degree':degree,
               'gamma': gamma}
svc = SVC()
svc_random = RandomizedSearchCV(estimator = svc, param_distributions = random_grid, n_iter =200, cv = 3, verbose=2, random_state=4872, n_jobs = -1)
svc_random.fit(X_train_smote, y_train_smote)

print(svc_random.best_params_)
params = svc_random.best_params_

In [ ]:
y_pred_svm = svc_random.predict(X_val_stand)
print(classification_report(y_val, y_pred_svm))

In [ ]:
out2 = pd.DataFrame(svc_random.cv_results_)

xlabel_names = ['C',
               'kernel',
               'degree',
               'gamma']

fig, axs = plt.subplots(2,2, figsize=(20,10))

axs[0,0].scatter(out2['param_C'], out2['mean_test_score'], c='blue');
axs[0,0].set_title('C')

axs[0,1].scatter(out2['param_kernel'], out2['mean_test_score'], c='blue');
axs[0,1].set_title('kernel')

axs[1,0].scatter(out2['param_degree'], out2['mean_test_score'], c='blue');
axs[1,0].set_title('degree')

axs[1,1].scatter(out2['param_gamma'], out2['mean_test_score'], c='blue');
axs[1,1].set_title('gamma')

for ax in axs.flat: ax.set(ylabel='r_squared')

The rbf kernel is clearly the best. This means we nog longer need the degree-parameter, since this is only used by the polynomial kernel. gamma around 0.2 perform best, and for C values between 7 and 8.

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


C = [round(x,5) for x in np.linspace(start = 7, stop = 8, num = 20)]
kernel = ['rbf']
gamma = [round(x,5) for x in np.linspace(start = 0.15, stop = 0.25, num = 20)]

random_grid = {'C': C,
               'kernel': kernel,
               'gamma': gamma}
svc = SVC()
svc_grid = GridSearchCV(estimator = svc, param_grid = random_grid,  cv = 3, verbose=2,  n_jobs = -1)

isotonic_svm = CalibratedClassifierCV(svc_grid, cv=3, method='isotonic')
isotonic_svm.fit(X_train_smote, y_train_smote)



In [ ]:
y_pred_svm = isotonic_svm.predict(X_val_stand)
print(classification_report(y_val, y_pred_svm))

In [ ]:
preds_svm = isotonic_svm.predict_proba(X_val_stand)[:,1]


fpr, tpr, threshold = metrics.roc_curve(y_val, preds)
fpr_p, tpr_p, threshold = metrics.roc_curve(y_val, preds_poly)
fpr_k, tpr_k, threshold = metrics.roc_curve(y_val, preds_knn)
fpr_g, tpr_g, threshold = metrics.roc_curve(y_val, preds_gbm)
fpr_s, tpr_s, threshold = metrics.roc_curve(y_val, preds_svm)

roc_auc = metrics.auc(fpr, tpr)
roc_auc_p = metrics.auc(fpr_p, tpr_p)
roc_auc_k = metrics.auc(fpr_k, tpr_k)
roc_auc_g = metrics.auc(fpr_g, tpr_g)
roc_auc_s = metrics.auc(fpr_s, tpr_s)


plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'logistic: AUC =  %0.2f' % roc_auc)
plt.plot(fpr_p, tpr_p, 'g', label = 'polynomial AUC = %0.2f' % roc_auc_p)
plt.plot(fpr_k, tpr_k, 'r', label = 'knn AUC = %0.2f' % roc_auc_k)
plt.plot(fpr_g, tpr_g, 'y', label = 'gbm AUC = %0.2f' % roc_auc_g)
plt.plot(fpr_s, tpr_s, 'grey', label = 'svm AUC = %0.2f' % roc_auc_s)

plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()